In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
               'price':'{:.2f}','buy_p':'{:.2f}','sell_p':'{:.2f}','diff':'{:.2f}',
               'ppu':'{:.4f}',
               'net':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}',
               'buy_amt':'{:,.2f}','cost_amt':'{:,.2f}','sell_amt':'{:,.2f}','profit':'{:,.2f}',
               'div_amt':'{:,.2f}','ttl_amt':'{:,.2f}','pct':'{:,.2f}',
               'sales_prf':'{:,.2f}','dividend':'{:,.2f}','net_div':'{:,.2f}','total_prf':'{:,.2f}',    
               'qty':'{:,}','number':'{:,}','days':'{:,}'
              }
pd.set_option("display.max_rows", None)

In [2]:
kind = 'SBD'
div_year = 2021
div_quarter = 1
sell_year = 2021

### Stocks that are sold on specified year

In [3]:
sql = '''
SELECT T.name, qty, S.price AS sell_p, B.price AS buy_p, S.price - B.price AS diff,
(S.net - B.net) AS sales_prf, B.net AS cost_amt,
ROUND((S.net-B.net)/B.net*36500/DATEDIFF(S.date,B.date),2) AS pct,
B.date AS buy_date, S.date AS sell_date,  DATEDIFF(S.date,B.date) AS days,
B.id AS bid, S.id AS sid, B.kind
FROM sells S
JOIN buys B ON S.buy_id = B.id 
JOIN stocks T ON B.stock_id = T.id 
WHERE kind = '%s'
ORDER BY T.name, S.date'''
sql = sql % (kind)
print(sql)


SELECT T.name, qty, S.price AS sell_p, B.price AS buy_p, S.price - B.price AS diff,
(S.net - B.net) AS sales_prf, B.net AS cost_amt,
ROUND((S.net-B.net)/B.net*36500/DATEDIFF(S.date,B.date),2) AS pct,
B.date AS buy_date, S.date AS sell_date,  DATEDIFF(S.date,B.date) AS days,
B.id AS bid, S.id AS sid, B.kind
FROM sells S
JOIN buys B ON S.buy_id = B.id 
JOIN stocks T ON B.stock_id = T.id 
WHERE kind = 'SBD'
ORDER BY T.name, S.date


In [6]:
sells = pd.read_sql(sql, conpf)
sells.sort_values(['sell_date','name'],ascending=[False,True]).style.format(format_dict)

,name,qty,sell_p,buy_p,diff,sales_prf,cost_amt,pct,buy_date,sell_date,days,bid,sid,kind
1,BKI,300,300.00,275.00,25.00,"7,117.93","82,682.73",523.70,2021-05-18,2021-05-24,6,897,716,SBD
3,SF,"20,000",5.45,5.20,0.25,"4,528.23","104,230.35",79.29,2021-02-17,2021-03-09,20,814,659,SBD
2,SF,"20,000",5.25,5.00,0.25,"4,545.95","100,221.49",413.90,2021-02-11,2021-02-15,4,808,646,SBD
0,BGC,"10,000",10.80,9.55,1.25,"12,049.26","95,711.53",510.56,2020-05-13,2020-05-22,9,713,572,SBD


In [17]:
mask_name = sells.name == 'BGC'
filter = sells[mask_name]
cols = 'name qty sell_p buy_p diff sales_prf cost_amt pct buy_date sell_date days bid sid kind'.split()
filter[cols].style.format(format_dict)

,name,qty,sell_p,buy_p,diff,sales_prf,cost_amt,pct,buy_date,sell_date,days,bid,sid,kind
0,BGC,"10,000",10.80,9.55,1.25,"12,049.26","95,711.53",510.56,2020-05-13,2020-05-22,9,713,572,SBD


In [19]:
mask_kind = sells.kind == 'SBD'
filter = sells[mask_kind]
filter[cols]

,name,qty,sell_p,buy_p,diff,sales_prf,cost_amt,pct,buy_date,sell_date,days,bid,sid,kind
0,BGC,10000,10.8,9.55,1.25,12049.26,95711.53,510.56,2020-05-13,2020-05-22,9,713,572,SBD
1,BKI,300,300.0,275.00,25.00,7117.93,82682.73,523.70,2021-05-18,2021-05-24,6,897,716,SBD


In [20]:
file_name = 'SELL-B4-DIV.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

('../data/SELL-B4-DIV.csv',
 '\\Users\\User\\iCloudDrive\\SELL-B4-DIV.csv',
 '\\Users\\User\\Dropbox\\SELL-B4-DIV.csv')

In [21]:
filter.dtypes

name          object
qty            int64
sell_p       float64
buy_p        float64
diff         float64
sales_prf    float64
cost_amt     float64
pct          float64
buy_date      object
sell_date     object
days           int64
bid            int64
sid            int64
kind          object
dtype: object

In [26]:
colt = 'name qty sell_p buy_p diff sales_prf cost_amt pct buy_date sell_date days'.split()

In [27]:
filter[colt].to_csv(output_file)
filter[colt].to_csv(data_file)
filter[colt].to_csv(box_file)

In [10]:
filter[colt].to_csv(output_file,mode='a',header=False)
filter[colt].to_csv(data_file,mode='a',header=False)
filter[colt].to_csv(box_file,mode='a',header=False)